In [15]:
import pandas as pd
import numpy as np
import math
import subprocess
from datetime import date, time, datetime, timedelta
from IPython.display import display
import calendar
import bandit.main as bd

In [16]:
def df_bandit_class(df, classname):
    dfgb = df.groupby([classname])
    dfgb = dfgb['Quality'].agg(['count','mean', 'var']).reset_index()
    dfgb['total'] = dfgb['count'].sum()
    dfgb['donext'] = dfgb.apply(lambda x: bd.get_tuned_ucb(x, 'mean', 'var'), axis=1).fillna(999)
    foo = dfgb.sort_values('donext', ascending=False)
    return foo

In [17]:
def df_search(df_in, cohort):
    if not cohort in df_in:
        return df_in

    df_class = df_bandit_class(df_in, cohort)
    display(df_class)
    next_class = df_class.iloc[0, 0]
    if df_in['Complete'].isnull().sum() > 0:
        df_filtered = df_in[df_in[cohort] == next_class]
    else:
        df_filtered = df_in

    return df_filtered

In [18]:
def rate_calc(df):
    pedal_time = df['Goal'] * df['seconds']
    rest_time = (df['Goal'] - 1) * 15
    total_time = pedal_time + rest_time
    rate = pedal_time / total_time
    score = pedal_time * rate
    return int(score)

In [19]:
currDate = datetime.today()
goal_list = [[x, y] for x in range(1,15) for y in [72,76,80,84,88]]
df = pd.DataFrame.from_records(goal_list).rename(index=str, columns={0: "Minutes", 1: "Rate"})
df['Date'] = currDate
df['Complete'] = np.nan
df = df[['Date','Minutes','Rate','Complete']]

In [20]:
df_excel = pd.read_excel(io='PunchGoals.xlsx')
foo = df_excel.groupby(['Minutes','Rate']).count().reset_index()
display(foo)
foo['remove'] = np.where(foo['Complete'] > 0, True, False)
foo = foo[['Minutes','Rate','remove']]
df = df.merge(foo,on=['Minutes','Rate'], how='left')
df = df[df['remove'] != True].drop(['remove'],axis=1)
df

,Minutes,Rate,Date,Complete
0,1,72,1,1
1,2,84,1,1
2,4,76,1,1
3,4,84,1,1
4,5,84,1,1
5,11,72,1,1


,Date,Minutes,Rate,Complete
1,2018-10-13 14:15:13.420340,1,76,NaN
2,2018-10-13 14:15:13.420340,1,80,NaN
3,2018-10-13 14:15:13.420340,1,84,NaN
4,2018-10-13 14:15:13.420340,1,88,NaN
5,2018-10-13 14:15:13.420340,2,72,NaN
6,2018-10-13 14:15:13.420340,2,76,NaN
7,2018-10-13 14:15:13.420340,2,80,NaN
9,2018-10-13 14:15:13.420340,2,88,NaN
10,2018-10-13 14:15:13.420340,3,72,NaN
11,2018-10-13 14:15:13.420340,3,76,NaN


In [21]:
subprocess.Popen('PunchGoals.xlsx', shell=True)

In [22]:
df = df.append(df_excel).reset_index(drop=True)

In [23]:
df['Value'] = df['Minutes'] * (df['Rate'] - 10)
val_df = bd.set_cohorts(df, 'Value')
df = df.merge(val_df, on='Value')
df['Score'] = np.where(df['Complete'] >= df['Minutes'], df['Value']**1.2, df['Value'] / 5)
df.loc[df['Complete'].isnull(), 'Score'] = np.NaN
df['Class'] = df['Minutes'].map(str) + '-' + df['Rate'].map(str)
df = bd.reduce(df,'Class')

0 1.0 0


In [24]:
df_quality = bd.get_quality(df['Score'])
df = df.join(df_quality)

In [25]:
df_step1 = df_search(df,'cohort1')
df_step1

,cohort1,count,mean,var,total,donext
0,0.0,4,0.581530,0.181072,6,0.916171
2,2.0,1,0.073434,NaN,6,0.742717
1,1.0,1,0.000000,NaN,6,0.669283


,Date,Minutes,Rate,Complete,Value,cohort1,cohort2,cohort3,Score,Class,Quality
0,2018-10-13 14:15:13.420340,1,76,NaN,66,0.0,0.0,0.0,NaN,1-76,NaN
1,2018-10-13 14:15:13.420340,1,80,NaN,70,0.0,0.0,0.0,NaN,1-80,NaN
2,2018-10-13 14:15:13.420340,1,84,NaN,74,0.0,0.0,1.0,NaN,1-84,NaN
3,2018-10-13 14:15:13.420340,1,88,NaN,78,0.0,0.0,1.0,NaN,1-88,NaN
4,2018-10-13 14:15:13.420340,2,72,NaN,124,0.0,0.0,1.0,NaN,2-72,NaN
5,2018-10-13 14:15:13.420340,2,76,NaN,132,0.0,0.0,2.0,NaN,2-76,NaN
6,2018-10-13 14:15:13.420340,2,80,NaN,140,0.0,0.0,2.0,NaN,2-80,NaN
7,2018-10-13 14:15:13.420340,2,88,NaN,156,0.0,1.0,3.0,NaN,2-88,NaN
8,2018-10-13 14:15:13.420340,3,72,NaN,186,0.0,1.0,3.0,NaN,3-72,NaN
9,2018-10-13 14:15:13.420340,3,76,NaN,198,0.0,1.0,4.0,NaN,3-76,NaN


In [26]:
df_step2 = df_search(df_step1,'cohort2')
df_step2

,cohort2,count,mean,var,total,donext
2,2.0,2,0.930272,0.009724,4,1.346549
1,1.0,1,0.386096,NaN,4,0.974801
0,0.0,1,0.079480,NaN,4,0.668185


,Date,Minutes,Rate,Complete,Value,cohort1,cohort2,cohort3,Score,Class,Quality
14,2018-10-13 14:15:13.420340,4,80,NaN,280,0.0,2.0,6.0,NaN,4-80,NaN
15,2018-10-13 14:15:13.420340,4,88,NaN,312,0.0,2.0,7.0,NaN,4-88,NaN
16,2018-10-13 14:15:13.420340,5,72,NaN,310,0.0,2.0,7.0,NaN,5-72,NaN
17,2018-10-13 14:15:13.420340,5,76,NaN,330,0.0,2.0,8.0,NaN,5-76,NaN
18,2018-10-13 14:15:13.420340,5,80,NaN,350,0.0,2.0,8.0,NaN,5-80,NaN
68,2018-10-10 00:00:00.000000,4,76,4.0,264,0.0,2.0,6.0,805.238836,4-76,0.860544
69,2018-10-13 00:00:00.000000,4,84,4.0,296,0.0,2.0,7.0,923.740645,4-84,1.000000


In [27]:
df_step3 = df_search(df_step2,'cohort3')
df_step3

,cohort3,count,mean,var,total,donext
2,8.0,0,NaN,NaN,2,999.000000
1,7.0,1,1.000000,NaN,2,1.416277
0,6.0,1,0.860544,NaN,2,1.276821


,Date,Minutes,Rate,Complete,Value,cohort1,cohort2,cohort3,Score,Class,Quality
17,2018-10-13 14:15:13.420340,5,76,NaN,330,0.0,2.0,8.0,NaN,5-76,NaN
18,2018-10-13 14:15:13.420340,5,80,NaN,350,0.0,2.0,8.0,NaN,5-80,NaN


In [28]:
df_bandit_class(df_step3, 'Class')

,Class,count,mean,var,total,donext
0,5-76,0,NaN,NaN,0,999.0
1,5-80,0,NaN,NaN,0,999.0
